<a href="https://colab.research.google.com/github/HimanshuSahoo/Python-for-Data-Scientist/blob/main/Lesson_35_Hand_written_digit_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import keras
import numpy
import keras.models

In [ ]:
from keras.datasets import mnist
from keras.models import sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend  as K
from keras.models import Sequential

In [ ]:
import logging, os
logging.basicConfig(level=logging.INFO)

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(x_train.shape, y_train.shape)

(60000, 28, 28) (60000,)


In [ ]:
from keras import utils as np_utils

In [ ]:
x_train=x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test=x_test.reshape(x_test.shape[0], 28, 28, 1)

num_classes=10
input_shape = (28,28,1)

y_train=keras.utils.np_utils.to_categorical(y_train, num_classes)
y_test=keras.utils.np_utils.to_categorical(y_test, num_classes)
x_train=x_train.astype('float32')
x_test=x_test.astype('float32')

x_train/=255
x_test/=255

print('X train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

X train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [ ]:
batch_size = 128
num_classes = 10
epochs = 10
model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3),activation = 'relu', input_shape = input_shape))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(256,activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation = 'softmax'))
model.compile(loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])
## after categorical_crossentropy optimizer = keras.optimizers.Adadelta(), to be added which is giving error


In [ ]:
hist = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose = 1, validation_data=(x_test, y_test))
print("The model has successfully trained")
model.save('mnist.h5')
print("saving the model as mnist.h5")

Epoch 1/10
469/469 [==============================] - 25s 53ms/step - loss: 0.2481 - accuracy: 0.9249 - val_loss: 0.0774 - val_accuracy: 0.9758
Epoch 2/10
469/469 [==============================] - 28s 59ms/step - loss: 0.0956 - accuracy: 0.9718 - val_loss: 0.0544 - val_accuracy: 0.9816
Epoch 3/10
469/469 [==============================] - 28s 59ms/step - loss: 0.0718 - accuracy: 0.9783 - val_loss: 0.0467 - val_accuracy: 0.9846
Epoch 4/10
469/469 [==============================] - 28s 59ms/step - loss: 0.0606 - accuracy: 0.9814 - val_loss: 0.0454 - val_accuracy: 0.9853
Epoch 5/10
469/469 [==============================] - 28s 59ms/step - loss: 0.0527 - accuracy: 0.9845 - val_loss: 0.0453 - val_accuracy: 0.9855
Epoch 6/10
469/469 [==============================] - 27s 59ms/step - loss: 0.0482 - accuracy: 0.9856 - val_loss: 0.0402 - val_accuracy: 0.9860
Epoch 7/10
469/469 [==============================] - 28s 59ms/step - loss: 0.0449 - accuracy: 0.9867 - val_loss: 0.0381 - val_accuracy:

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print('test loss:', score[0])
print('test accuracy:', score[1])

test loss: 0.036420661956071854
test accuracy: 0.988099992275238


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from keras.models import load_model
from tkinter import *
import tkinter as tk
import win32gui
from PIL import ImageGrab , Image

model = load_model('mnist.h5')

def predict_digit(img):
    img=img.resize((28,28))
    img=img.conver('L')
    img=np.array(img)
    img=img.reshape(1,28,28,1)
    img=img/255
    res=model.predict([img])[0]
    return np.argmax(res), max(res)
class App(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)
        self.x=self.y=0
        
        self.canvas=tk.Canvas(self,width=300,height=300,bg="white",cursor="cross")
        self.label=tk.Label(self,text="Thinking..", font=("Helvetica",48))
        self.classify_btn=tk.Button(self,text="Recognise", command=self.classify_handwriting)
        self.button_clear = tk.Button(self, text="Clear", command=self.clear_all)
        
        self.canvas.grid(row=0, column=0, pady=2, sticky=W,)
        self.label.grid(row=0, column=1, pady=2, padx=2)
        self.classify_btn.grid(row=1, column=1, pady=2, padx=2)
        self.button_clear.grid(row=1, column=0, pady=2)
        
        self.canvas.bind("<B1-Motion>",self.draw_lines)
    
    def clear_all(self):
        self.canvas.delete("all")
    
    def classify_handwriting(self):
        HWND = self.canvas.winfo_id()
        rect = win32gui.GetWindowRect(HWND)
        im = ImageGrab.grab(rect)
        
        digit,acc = predict_digit(im)
        self.label.configure(text=str(digit)+', '+str(int(acc*100))+'%')
        
    def draw_lines(self, event):
        self.x = event.x
        self.y = event.y
        r=8
        self.canvas.create_oval(self.x-r, self.y-r, self.x+r, self.y+r, fill = black)
        
app = App()
mainloop()
